# Google Colab

- As we are working with more and more data, we may need GPU computing for quicker processing.
- This lecture note shows how we can capitalize on the free GPU computing provided by Google Colab and speed up the Chinese word segmentation of `ckip-transformers`.

## Setup Google Drive

- Create a working directory under your Google Drive, named `ENC2045_DEMO_DATA`.
- Save the corpus files needed in that Google Drive directory.
- We can access the files on our Google Drive from Google Colab. This can be useful when you need to load your own data in Google Colab.

## Run Notebook in Google Colab

- Click on the button on top of the lecture notes website to open this notebook in Google Colab.

## Setting Google Colab Environment

- GPU Setting:
    - [Runtime] -> [Change runtime type]
    - For [Hardware accelerator], choose [GPU]

In [9]:
!nvidia-smi

Wed Mar 24 11:27:29 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P0    31W /  70W |  10734MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

- Install modules that are not installed in the current Google Colab

In [1]:
## Google Drive Setting
!pip install ckip-transformers

     |████████████████████████████████| 2.0MB 12.8MB/s 
     |████████████████████████████████| 890kB 51.9MB/s 
     |████████████████████████████████| 3.2MB 56.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=becb597660b6f2d76afbd63dc0edb32385a77c7c6318a4ace6ad68116035b953
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


- Mount Our Google Drive
    - After we run the above codes, get the authorization code and copy-paste it to the box.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


- Change Colab working directory to the `ENC2045_demo_data` of the Google Drive

In [3]:
import os
os.chdir('/content/drive/MyDrive/ENC2045_demo_data')
print(os.getcwd())


/content/drive/MyDrive/ENC2045_demo_data


## Try `ckip-transformers` with GPU

### Initialize the `ckip-transformers`

In [4]:
import ckip_transformers
from ckip_transformers.nlp import CkipWordSegmenter, CkipPosTagger
# Initialize drivers
ws_driver = CkipWordSegmenter(level=3, device=0)
pos_driver = CkipPosTagger(level=3, device=0)


In [5]:
def my_tokenizer(doc):
    # `doc`: a list of corpus documents (each element is a document long string)
    cur_ws = ws_driver(doc, use_delim = True, delim_set='\n')
    cur_pos = pos_driver(cur_ws)
    doc_seg = [[(x,y) for (x,y) in zip(w,p)]  for (w,p) in zip(cur_ws, cur_pos)]
    return doc_seg

### Tokenization Chinese Texts

In [6]:
import pandas as pd

df = pd.read_csv('dcard-top100.csv')
df.head()
corpus = df['content']
corpus[:10]

0    部分回應在B117 \n謝謝各位的留言，我都有看完\n好的不好的，我都接受謝謝大家🙇‍♀️\...
1    https://i.imgur.com/REIEzSd.jpg\n\n身高195公分的男大生...
2    看過這麼多在Dcard、PTT上的感情渣事和創作文\n從沒想過如此荒謬像八點檔的事情居然會發...
3    剛剛吃小火鍋，跟店員說不要金針菇（怕卡牙縫），於是店員幫我換其他配料..…\n\n沒想到餐一...
4    已經約好見面，到了當天晚上七點半才回，我是被耍了嗎 \n如下圖\n\n\nhttps://i...
5    嗨！巨砲哥 答應你的文來了😆\n這是一段與約砲小哥哥談心的奇幻旅程\n\n可憐的我情人節當天...
6    https://i.imgur.com/HCTwyAH.jpg\n（圖片非本人）\n今天逛街...
7    https://i.imgur.com/RWJLK2v.jpg\n\n因為馬鞍很寬\n想請問...
8    手機排版請見諒😖🙏🏻（圖多）\n先說這不是我第一次訂購訂製蛋糕\n也了解訂製蛋糕不可能跟圖上...
9    https://i.imgur.com/6Yk9etg.jpg\n想在這裡問大家有沒有接到這...
Name: content, dtype: object

In [7]:
%%time
corpus_seg = my_tokenizer(corpus)

Inference: 100%|██████████| 10/10 [01:39<00:00,  9.93s/it]


CPU times: user 2min 4s, sys: 2min 4s, total: 4min 9s
Wall time: 4min 8s


In [8]:
corpus_seg[0][:50]

[('部分', 'Neqa'),
 ('回應', 'VC'),
 ('在', 'P'),
 ('B117 \n', 'FW'),
 ('謝謝', 'VJ'),
 ('各位', 'Nh'),
 ('的', 'DE'),
 ('留言', 'Na'),
 ('，', 'COMMACATEGORY'),
 ('我', 'Nh'),
 ('都', 'D'),
 ('有', 'D'),
 ('看完', 'VC'),
 ('\n', 'WHITESPACE'),
 ('好', 'VH'),
 ('的', 'DE'),
 ('不', 'D'),
 ('好', 'VH'),
 ('的', 'T'),
 ('，', 'COMMACATEGORY'),
 ('我', 'Nh'),
 ('都', 'D'),
 ('接受', 'VC'),
 ('謝謝', 'VJ'),
 ('大家', 'Nh'),
 ('🙇', 'FW'),
 ('\u200d♀️\n', 'DASHCATEGORY'),
 ('\n', 'WHITESPACE'),
 ('\n', 'WHITESPACE'),
 ('（', 'PARENTHESISCATEGORY'),
 ('第三', 'Neu'),
 ('次', 'Nf'),
 ('更新', 'VC'),
 ('在', 'P'),
 ('這邊', 'Ncd'),
 ('）', 'PARENTHESISCATEGORY'),
 ('\n', 'WHITESPACE'),
 ('B258 ', 'FW'),
 ('這邊', 'Ncd'),
 ('也', 'D'),
 ('有', 'V_2'),
 ('講到', 'VE'),
 ('怎麼', 'D'),
 ('逃生', 'VA'),
 ('\n', 'WHITESPACE'),
 ('很多', 'Neqa'),
 ('人', 'Na'),
 ('好奇', 'VH'),
 ('我', 'Nh'),
 ('是', 'SHI')]